# Proper code

## Shapley function

In [10]:
import numpy as np
import openturns as ot
import pandas as pd

def ShapleyPerm(method, m, model, Xall, Xcond, d, Nv, No, Ni=3):
    """
    """
    if (method == 'exact'):
        perms_tool = ot.KPermutations(d, d)
        perms = perms_tool.generate()

        m = perms.getSize() # number of permutation
    else:
        perms = np.zeros((m,d), dtype = np.int64)
        for i in range(m):
            perms[i] = np.random.permutation(d) # random permutation
    
    #------------------------------
    # Creation of the design matrix
    #------------------------------
    
    X = np.zeros((Nv+m*(d-1)*No*Ni, d)) 
    X[:Nv,:] = Xall(Nv)

    for p in range(m):
    
        pi = perms[p]
        pi_sorted = np.argsort(pi)
    
        for j in range(1,d):
        
            Sj = pi[:j] # set of the 0st-(j-1)th elements in pi      
            Sjc = pi[j:] # set of the jth-dth elements in pi
        
            xjcM = Xcond(No,Sjc,None,None)# sampled values of the inputs in Sjc

            for l in range(No):
                
                xjc = xjcM[l,:]
            
                # sample values of inputs in Sj conditional on xjc
                xj = Xcond(Ni, Sj, Sjc, xjc)
                xx = np.concatenate((xj, np.ones((Ni,1))*xjc), axis = 1)
                ind_inner = Nv + p*(d-1)*No*Ni + (j-1)*No*Ni + l*Ni
                X[ind_inner:(ind_inner + Ni),:] = xx[:,pi_sorted]
    
    #-----------------------
    # Calcul of the response
    #-----------------------
    
    y = model(X)

    #-----------------------------------------------------------------
    # Initialize Shapley, main and total Sobol effects for all players
    #-----------------------------------------------------------------
    
    Sh = np.zeros(d)
    Vsob = np.zeros(d)
    Tsob = np.zeros(d)
    
    nV = np.zeros(d) # number of samples used to estimate V1,...,Vd
    nT = np.zeros(d) # number of samples used to estimate T1,...,Td
    
    #----------------
    # Estimate Var[Y]
    #----------------
    
    Y = y[:Nv]
    y = y[Nv:]
    EY = np.mean(Y)
    VarY = np.var(Y)

    #-----------------------------------------------
    # Estimate Shapley, main and total Sobol effects
    #-----------------------------------------------
    
    cVar = np.zeros(No)

    for p in range(m):
    
        pi = perms[p]
        prevC = 0
    
        for j in range(d):
            if (j == (d-1)):
                Chat = VarY
                delta = Chat - prevC
                Vsob[pi[j]] = Vsob[pi[j]] + prevC # first order effect
                nV[pi[j]] = nV[pi[j]] + 1
            else:
                for l in range(No):
                    Y = y[:Ni]
                    y = y[Ni:]
                    cVar[l] = np.var(Y)
                Chat = np.mean(cVar)
                delta = Chat - prevC
      
            Sh[pi[j]] = Sh[pi[j]] + delta
        
            prevC = Chat
        
            if (j == 0):
                Tsob[pi[j]] = Tsob[pi[j]] + Chat # Total effect
                nT[pi[j]] = nT[pi[j]] + 1
    
    Sh = Sh / m / VarY
    
    if (method == 'exact'):
        Vsob = Vsob / (m/d) / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / (m/d) / VarY # averaging by number of permutations with j=1 
    else:
        Vsob = Vsob / nV / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / nT / VarY # averaging by number of permutations with j=1 
    
    col = ['X' + str(i) for i in np.arange(d)+1]
    effects = pd.DataFrame(np.array([Sh,Vsob,Tsob]), index = ['Shapley effects', 'First order Sobol', 'Total Sobol'], columns = col)

    return effects

In [12]:
def condMVN(mean, cov, dependent_ind, given_ind, X_given):
    """ Returns conditional mean and variance of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cov = np.array(cov)
    
    B = cov.take(dependent_ind, axis = 1)
    B = B[dependent_ind]
    
    C = cov.take(dependent_ind, axis = 1)
    C = C[given_ind]
    
    D = cov.take(given_ind, axis = 1)
    D = D[given_ind]
    
    CDinv = np.dot(np.transpose(C),np.linalg.inv(D))
    
    condMean = mean[dependent_ind] + np.dot(CDinv,(X_given - mean[given_ind]))
    condVar = B - np.dot(CDinv,C)
    condVar = ot.CovarianceMatrix(condVar)
    
    return condMean,condVar

#condMVN(moyenne, cov, dependent_ind = np.array([1,0]), given_ind= np.array([2]), X_given= np.array([10]))

In [34]:
#condMVN(moyenne, cov, dependent_ind = np.array([1,0]), given_ind= np.array([2]), X_given= np.array([10]))

#dependent_ind = np.array([1,0])
#dependent_ind

#given_ind= np.array([2])
#given_ind

##X_given= np.array([10])
#X_given

Z=condMVN(moyenne, cov, dependent_ind = np.array([1,0]), given_ind= np.array([2]), X_given= np.array([10]))
Z

(array([4.5, 0. ]),
 class=CovarianceMatrix dimension=2 implementation=class=MatrixImplementation name=Unnamed rows=2 columns=2 values=[0.19,0,0,1])

## Function for conditional gaussian vector

### Calcul of conditional mean and variance

In [14]:
def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cond_mean,cond_var = condMVN(mean, cov, dependent_ind, given_ind, X_given)
    distribution = ot.Normal(cond_mean,cond_var)
    return distribution.getSample(n)

## Evaluation Shapley effects on linear gaussian model

### Ni = 3

In [25]:
def gaussian_model(X):
    return np.sum(X,1)

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 3

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 3

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.180492  0.388887  0.430621
First order Sobol  0.407860  0.879894  0.919761
Total Sobol        0.067213  0.013164  0.051871


Random method 
                         X1        X2        X3
Shapley effects    0.179297  0.385052  0.435651
First order Sobol  0.398373  0.879094  0.916758
Total Sobol        0.070965  0.013132  0.051876



### Ni = 100

In [26]:
def gaussian_model(X):
    return np.sum(X,1)

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 100

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 100

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.110139  0.415858  0.474003
First order Sobol  0.126404  0.820864  0.878634
Total Sobol        0.101887  0.019368  0.077352


Random method 
                         X1        X2        X3
Shapley effects    0.107475  0.412271  0.480254
First order Sobol  0.117732  0.820460  0.877827
Total Sobol        0.102682  0.019599  0.077752

